In [2]:
from ocpmodels.datasets import TrajectoryLmdbDataset
import numpy as np
from  functions import *
%run  ocp-tutorial.ipynb

In [3]:
checkpoint = "eq2_31M_ec4_allmd.pt"
src = "./03-lmdb/ads_ads-3s_ns3"
max_epochs = 30
batch_size = 3
eval_batch_size = 3
element_c = "bs"+str(batch_size)+"_"
train_src = src + "/train"
val_src =  src + "/val"
id_name = get_filename_from_path(checkpoint)
os.makedirs("./config", exist_ok=True)
os.makedirs("./train_log", exist_ok=True)
config_path = "./config/"+ element_c+get_filename_from_path(src)+".yml"
train_txt = "./train_log/"+ element_c+get_filename_from_path(src)+ "_" +str(max_epochs) +".txt"
train_dataset = TrajectoryLmdbDataset({"src": train_src})

energies = []
forces = []
for data in train_dataset:
  energies.append(data.y)
  for i in range(3):
    for j in range(len(data.force)):
      forces.append(data.force[j][i])

target_mean = np.mean(energies)
target_std = np.std(energies)
grad_target_mean = np.mean(forces)
grad_target_std = np.std(forces)
len_train = len(train_dataset)
target_mean,grad_target_mean,grad_target_std,len_train

e:\soft\path\anaconda\envs\ocp\lib\site-packages\IPython\core\interactiveshell.py:3505: UserWarning: TrajectoryLmdbDataset is deprecated and will be removed in the future.Please use 'LmdbDataset' instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


(-1.695643508367349, -0.0135889, 0.07754764, 343)

In [4]:
yml = generate_yml_config(checkpoint, config_path,
                   delete=['slurm', 'cmd', 'logger',  'model_attributes',
                           'dataset', 'test_dataset', 'val_dataset'],
                   update={'gpus': 1,
                           'task.dataset': 'lmdb',
                           'optim.eval_every': int(f'{len_train}'),
                           'optim.max_epochs': int(f'{max_epochs}'),
                           'optim.num_workers' : 0,
                           'optim.batch_size': int(f'{batch_size}') ,
                           'optim.eval_batch_size' : int(f'{eval_batch_size}'),
                           # Train data
                           'dataset.train.src': "%s" % train_src,
                           'dataset.train.normalize_labels': True,
                           'dataset.train.target_mean': float(f'{target_mean}'),
                           'dataset.train.target_std': float(f'{target_std}'),
                           'dataset.train.grad_target_mean': float(f'{grad_target_mean}'),
                           'dataset.train.grad_target_std': float(f'{grad_target_mean}'),
                           # val data
                           'dataset.val.src': "%s" % val_src,
                          })

yml

WindowsPath('d:/000-SI/01-train/config/bs3_ads_ads-3s_ns3.yml')

In [5]:
import time
t0 = time.time()
! python {ocp_main()} --mode train --config-yml $yml --checkpoint $checkpoint \
  --run-dir fine-tuning  --identifier $id_name --amp > $train_txt 2>&1 
print(f'Elapsed time = {time.time() - t0:1.1f} seconds')

Elapsed time = 1289.9 seconds
